In [257]:
import random

# Define the fitness function
def fitness_function(x):
    return (-x**2)/10 + 3*x

# Function to generate a random chromosome
def generate_chromosome():
    return [random.randint(0, 1) for _ in range(5)]

# Function to decode chromosome to x value
def decode_chromosome(chromosome):
    x = 0
    for bit in chromosome:
        x = (x << 1) | bit
    return x

# Function to perform crossover between two chromosomes
def crossover(chromosome1, chromosome2):
    crossover_point = random.randint(1, len(chromosome1) - 1)
    new_chromosome1 = chromosome1[:crossover_point] + chromosome2[crossover_point:]
    new_chromosome2 = chromosome2[:crossover_point] + chromosome1[crossover_point:]
    return new_chromosome1, new_chromosome2

# Function to perform mutation on a chromosome
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)
    chromosome[mutation_point] = 1 - chromosome[mutation_point]  # Flip the bit
    return chromosome

# Initialize the population
population_size = 10
population = [generate_chromosome() for _ in range(population_size)]

# Main loop
generation = 1
while True:
    # Calculate fitness values for each chromosome
    fitness_values = [fitness_function(decode_chromosome(chromosome)) for chromosome in population]

    # Check termination condition
    best_fitness = max(fitness_values)
    if best_fitness >= 0.9 * max(fitness_values):
        break

    # Selection - Roulette wheel selection
    selection_probabilities = [fitness / sum(fitness_values) for fitness in fitness_values]
    selected_indices = random.choices(range(population_size), weights=selection_probabilities, k=population_size)

    # Create the new population through crossover and mutation
    new_population = []
    for i in range(0, population_size, 2):
        chromosome1 = population[selected_indices[i]]
        chromosome2 = population[selected_indices[i+1]]
        chromosome1, chromosome2 = crossover(chromosome1, chromosome2)

        if generation % 3 == 0:  # Apply mutation every 3 generations
            chromosome1 = mutate(chromosome1)
            chromosome2 = mutate(chromosome2)

        new_population.extend([chromosome1, chromosome2])

    population = new_population
    generation += 1

# Print the result
best_chromosome = population[fitness_values.index(max(fitness_values))]
best_x = decode_chromosome(best_chromosome)
best_fitness = fitness_function(best_x)

print("Best Chromosome:", best_chromosome)
print("Best x:", best_x)
print("Best Fitness:", best_fitness)


Best Chromosome: [0, 1, 1, 1, 1]
Best x: 15
Best Fitness: 22.5


In [193]:
import random

locations = ["MM Alam Road", "Johar Town", "Shahdara", "DHA Phase 6", "Wapda Town", "Askari 10", "Allama Iqbal Town", "Mall Road"]
num_locations = len(locations)
population_size = 100
mutation_rate = 0.01
num_generations = 100

distance_matrix = [[0, 12, 25, 8, 18, 14, 23, 10],
                   [12, 0, 32, 15, 27, 20, 7, 22],
                   [25, 32, 0, 19, 9, 30, 28, 17],
                   [8, 15, 19, 0, 14, 12, 21, 5],
                   [18, 27, 9, 14, 0, 25, 13, 20],
                   [14, 20, 30, 12, 25, 0, 16, 11],
                   [23, 7, 28, 21, 13, 16, 0, 9],
                   [10, 22, 17, 5, 20, 11, 9, 0]]

def generate_population():
    population = []
    for _ in range(population_size):
        chromosome = random.sample(range(num_locations), num_locations)
        population.append(chromosome)
    return population

def calculate_distance(route):
    total_distance = 0
    for i in range(num_locations):
        from_location = route[i]
        to_location = route[(i + 1) % num_locations]
        total_distance += distance_matrix[from_location][to_location]
    return total_distance

def crossover(parent1, parent2):
    start = random.randint(0, num_locations - 1)
    end = random.randint(start + 1, num_locations)
    child = [-1] * num_locations
    child[start:end] = parent1[start:end]
    index = end
    for gene in parent2:
        if gene not in child:
            child[index % num_locations] = gene
            index += 1
    return child

def mutate(individual):
    for _ in range(2):
        pos1 = random.randint(0, num_locations - 1)
        pos2 = random.randint(0, num_locations - 1)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
def genetic_algorithm():
    population = generate_population()

    for generation in range(num_generations):
        fitness_scores = [1 / calculate_distance(chromosome) for chromosome in population]
        if any(fitness >= 0.9 for fitness in fitness_scores):
            best_individual = population[fitness_scores.index(max(fitness_scores))]
            return best_individual
        parents = random.choices(population, weights=fitness_scores, k=2)
        offspring = [crossover(parents[0], parents[1]) for _ in range(population_size)]
        for individual in offspring:
            if random.random() < mutation_rate:
                mutate(individual)
                
        population = offspring
    best_individual = min(population, key=calculate_distance)
    return best_individual

best_route = genetic_algorithm()

print("Best Route:")
for location in best_route:
    print(locations[location])
print("Total Distance:", calculate_distance(best_route))



Best Route:
Shahdara
Mall Road
Allama Iqbal Town
Askari 10
DHA Phase 6
MM Alam Road
Johar Town
Wapda Town
Total Distance: 110


In [201]:
import random

n = 8
population_size = 100
mutation_rate = 0.01
num_generations = 100

def generate_population():
    population = []
    for _ in range(population_size):
        chromosome = random.sample(range(1, n+1), n)
        population.append(chromosome)
    return population

def calculate_threats(configuration):
    threats = 0
    for i in range(n):
        for j in range(i+1, n):
            if abs(configuration[j] - configuration[i]) == abs(j - i):
                threats += 1
    return threats

def crossover(parent1, parent2):
    child = [-1] * n
    start = random.randint(0, n-1)
    end = random.randint(start + 1, n)
    child[start:end] = parent1[start:end]
    for i in range(n):
        if parent2[i] not in child:
            for j in range(n):
                if child[j] == -1:
                    child[j] = parent2[i]
                    break
    return child

def mutate(individual):
    for _ in range(2):
        pos1 = random.randint(0, n-1)
        pos2 = random.randint(0, n-1)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]

def genetic_algorithm():
    population = generate_population()

    for generation in range(num_generations):
        fitness_scores = [n - calculate_threats(chromosome) for chromosome in population]
        if n in fitness_scores:
            best_individual = population[fitness_scores.index(n)]
            return best_individual
        parents = random.choices(population, weights=fitness_scores, k=2)
        offspring = [crossover(parents[0], parents[1]) for _ in range(population_size)]
        for individual in offspring:
            if random.random() < mutation_rate:
                mutate(individual)
        population = offspring
    best_individual = max(population, key=lambda x: n - calculate_threats(x))
    return best_individual

best_configuration = genetic_algorithm()
print("Best Configuration:", best_configuration)


Best Configuration: [2, 5, 6, 3, 7, 4, 1, 8]
